# Chapter 17: Async Networking

**Networking and Protocols**

The `asyncio` module provides high-level APIs for asynchronous networking:
`open_connection()` for TCP clients and `start_server()` for TCP servers.
These use `StreamReader` and `StreamWriter` to abstract away the complexities
of non-blocking I/O, making it straightforward to handle thousands of
concurrent connections on a single thread.

## Async TCP Client with `open_connection()`

`asyncio.open_connection()` establishes a TCP connection and returns a
`(StreamReader, StreamWriter)` pair. The `StreamReader` provides async methods
for reading (`read()`, `readline()`, `readexactly()`), and the `StreamWriter`
provides `write()` and `drain()` for non-blocking sends.

Key pattern: `writer.write()` buffers data, then `await writer.drain()` ensures
the buffer is flushed to the network.

In [ ]:
import asyncio


async def async_echo_server_handler(
    reader: asyncio.StreamReader,
    writer: asyncio.StreamWriter,
) -> None:
    """Handle a single client connection: echo lines back."""
    addr = writer.get_extra_info("peername")
    print(f"[Server] Client connected from {addr}")

    while True:
        data: bytes = await reader.readline()
        if not data:
            break
        message: str = data.decode().strip()
        print(f"[Server] Received: {message!r}")

        response: bytes = f"ECHO: {message}\n".encode()
        writer.write(response)
        await writer.drain()  # Ensure data is sent

    print(f"[Server] Client {addr} disconnected")
    writer.close()
    await writer.wait_closed()


async def async_echo_client(
    host: str,
    port: int,
    messages: list[str],
) -> list[str]:
    """Connect to the echo server and exchange messages."""
    reader, writer = await asyncio.open_connection(host, port)
    print(f"[Client] Connected to {host}:{port}")

    responses: list[str] = []
    for msg in messages:
        # Send a line-terminated message
        writer.write(f"{msg}\n".encode())
        await writer.drain()

        # Read the echoed response
        data: bytes = await reader.readline()
        response: str = data.decode().strip()
        print(f"[Client] Sent: {msg!r}, Got: {response!r}")
        responses.append(response)

    writer.close()
    await writer.wait_closed()
    return responses


async def demo_client_server() -> None:
    """Run an async echo server and client."""
    # Start the server on a random port
    server = await asyncio.start_server(
        async_echo_server_handler, "127.0.0.1", 0
    )
    addr = server.sockets[0].getsockname()
    port: int = addr[1]
    print(f"Server started on 127.0.0.1:{port}")

    # Run the client
    messages = ["Hello async!", "Python networking", "Goodbye"]
    responses = await async_echo_client("127.0.0.1", port, messages)

    # Shut down the server
    server.close()
    await server.wait_closed()

    expected = [f"ECHO: {m}" for m in messages]
    print(f"\nAll responses correct: {responses == expected}")


await demo_client_server()

## StreamReader and StreamWriter API

The `StreamReader` and `StreamWriter` classes provide a clean abstraction
over raw sockets:

**StreamReader methods:**
- `read(n)` -- Read up to `n` bytes
- `readline()` -- Read until `\n` (or EOF)
- `readexactly(n)` -- Read exactly `n` bytes (raises `IncompleteReadError` if EOF)
- `readuntil(separator)` -- Read until a specific byte sequence

**StreamWriter methods:**
- `write(data)` -- Buffer data for sending (synchronous, non-blocking)
- `writelines(data_list)` -- Buffer multiple chunks
- `drain()` -- Await until the write buffer is flushed
- `close()` / `wait_closed()` -- Close the connection gracefully
- `get_extra_info(name)` -- Get transport info (e.g., `peername`, `sockname`)

In [ ]:
import asyncio


async def demonstrate_stream_api() -> None:
    """Show various StreamReader/StreamWriter methods."""

    async def server_handler(
        reader: asyncio.StreamReader,
        writer: asyncio.StreamWriter,
    ) -> None:
        """Server that uses different read methods."""
        # 1. readexactly: read a fixed number of bytes
        header: bytes = await reader.readexactly(4)
        print(f"[Server] readexactly(4): {header!r}")

        # 2. readline: read until newline
        line: bytes = await reader.readline()
        print(f"[Server] readline(): {line!r}")

        # 3. read: read up to N bytes
        chunk: bytes = await reader.read(100)
        print(f"[Server] read(100): {chunk!r}")

        # Send a multi-part response using writelines
        writer.writelines([
            b"HTTP/1.1 200 OK\r\n",
            b"Content-Type: text/plain\r\n",
            b"\r\n",
            b"Response body here",
        ])
        await writer.drain()

        # Get connection info
        peername = writer.get_extra_info("peername")
        sockname = writer.get_extra_info("sockname")
        print(f"[Server] peername={peername}, sockname={sockname}")

        writer.close()
        await writer.wait_closed()

    # Start server
    server = await asyncio.start_server(server_handler, "127.0.0.1", 0)
    port: int = server.sockets[0].getsockname()[1]

    # Client side
    reader, writer = await asyncio.open_connection("127.0.0.1", port)

    # Send data that matches the server's read pattern
    writer.write(b"HDR!")               # 4 bytes for readexactly
    writer.write(b"A line of text\n")   # For readline
    writer.write(b"Remaining data")     # For read
    await writer.drain()

    # Signal end of writing
    writer.write_eof()

    # Read the full response
    response: bytes = await reader.read(-1)  # Read until EOF
    print(f"\n[Client] Full response:\n{response.decode()}")

    writer.close()
    await writer.wait_closed()
    server.close()
    await server.wait_closed()


await demonstrate_stream_api()

## `asyncio.start_server` for TCP Servers

`asyncio.start_server()` creates a TCP server that calls a handler coroutine
for each new connection. Unlike threading-based servers, all connections are
handled on a single thread via the event loop. This scales efficiently to
thousands of simultaneous connections.

The server object supports `serve_forever()` for long-running services,
or can be used as an async context manager.

In [ ]:
import asyncio
from datetime import datetime


# Track connected clients
connected_clients: set[str] = set()


async def chat_handler(
    reader: asyncio.StreamReader,
    writer: asyncio.StreamWriter,
) -> None:
    """Handle a client in a simple chat-like server."""
    addr = writer.get_extra_info("peername")
    client_id: str = f"{addr[0]}:{addr[1]}"
    connected_clients.add(client_id)

    # Send welcome message
    welcome: str = f"Welcome! You are {client_id}. {len(connected_clients)} client(s) connected.\n"
    writer.write(welcome.encode())
    await writer.drain()

    try:
        while True:
            data: bytes = await reader.readline()
            if not data:
                break
            message: str = data.decode().strip()
            timestamp: str = datetime.now().strftime("%H:%M:%S")
            response: str = f"[{timestamp}] {client_id}: {message}\n"
            writer.write(response.encode())
            await writer.drain()
    except ConnectionResetError:
        pass
    finally:
        connected_clients.discard(client_id)
        writer.close()
        await writer.wait_closed()
        print(f"[Server] {client_id} disconnected ({len(connected_clients)} remaining)")


async def demo_chat_server() -> None:
    """Run a chat server with multiple concurrent clients."""
    server = await asyncio.start_server(chat_handler, "127.0.0.1", 0)
    port: int = server.sockets[0].getsockname()[1]
    print(f"Chat server running on 127.0.0.1:{port}")

    async def simulate_client(name: str, messages: list[str]) -> list[str]:
        """Simulate a chat client."""
        reader, writer = await asyncio.open_connection("127.0.0.1", port)

        # Read welcome message
        welcome: bytes = await reader.readline()
        print(f"  [{name}] {welcome.decode().strip()}")

        responses: list[str] = []
        for msg in messages:
            writer.write(f"{msg}\n".encode())
            await writer.drain()
            response: bytes = await reader.readline()
            resp_text: str = response.decode().strip()
            print(f"  [{name}] Got: {resp_text}")
            responses.append(resp_text)
            await asyncio.sleep(0.05)  # Simulate think time

        writer.close()
        await writer.wait_closed()
        return responses

    # Run 3 clients concurrently
    results = await asyncio.gather(
        simulate_client("Alice", ["Hello!", "How is everyone?"]),
        simulate_client("Bob", ["Hi there!", "Doing great!"]),
        simulate_client("Carol", ["Hey!", "Good to be here!"]),
    )

    print(f"\nTotal messages exchanged: {sum(len(r) for r in results)}")

    server.close()
    await server.wait_closed()


connected_clients.clear()
await demo_chat_server()

## Concurrent Connections with `asyncio.gather()`

`asyncio.gather()` runs multiple coroutines concurrently on the event loop.
This is ideal for making many network requests in parallel -- for example,
querying multiple API endpoints simultaneously or connecting to several
servers at once.

In [ ]:
import asyncio
import time


async def simulated_api_call(endpoint: str, delay: float) -> dict[str, str | float]:
    """Simulate an async API call with variable latency."""
    start: float = time.perf_counter()
    await asyncio.sleep(delay)  # Simulate network I/O
    elapsed: float = time.perf_counter() - start
    return {
        "endpoint": endpoint,
        "status": "ok",
        "latency": round(elapsed, 3),
    }


async def demo_concurrent_requests() -> None:
    """Show the performance difference between sequential and concurrent."""
    endpoints: list[tuple[str, float]] = [
        ("/api/users", 0.3),
        ("/api/products", 0.5),
        ("/api/orders", 0.2),
        ("/api/inventory", 0.4),
        ("/api/analytics", 0.6),
    ]

    # Sequential: await each one at a time
    print("=== Sequential Requests ===")
    start = time.perf_counter()
    sequential_results: list[dict[str, str | float]] = []
    for endpoint, delay in endpoints:
        result = await simulated_api_call(endpoint, delay)
        sequential_results.append(result)
    seq_time: float = time.perf_counter() - start

    for r in sequential_results:
        print(f"  {r['endpoint']}: {r['latency']}s")
    print(f"  Total: {seq_time:.3f}s\n")

    # Concurrent: gather all at once
    print("=== Concurrent Requests (asyncio.gather) ===")
    start = time.perf_counter()
    concurrent_results = await asyncio.gather(
        *[simulated_api_call(ep, delay) for ep, delay in endpoints]
    )
    con_time: float = time.perf_counter() - start

    for r in concurrent_results:
        print(f"  {r['endpoint']}: {r['latency']}s")
    print(f"  Total: {con_time:.3f}s")
    print(f"\nSpeedup: {seq_time / con_time:.1f}x")


await demo_concurrent_requests()

## Timeouts with `asyncio.wait_for()`

Network operations should always have timeouts to prevent indefinite hangs.
`asyncio.wait_for()` wraps any awaitable with a timeout: if the operation
does not complete within the specified duration, `asyncio.TimeoutError` is
raised and the wrapped task is cancelled.

In [ ]:
import asyncio


async def slow_network_call(name: str, delay: float) -> str:
    """Simulate a network call that may be slow."""
    await asyncio.sleep(delay)
    return f"{name} completed in {delay}s"


async def fetch_with_timeout(
    name: str,
    delay: float,
    timeout: float,
) -> str:
    """Fetch with a timeout, returning an error message on timeout."""
    try:
        result: str = await asyncio.wait_for(
            slow_network_call(name, delay),
            timeout=timeout,
        )
        return result
    except asyncio.TimeoutError:
        return f"{name} TIMED OUT after {timeout}s (needed {delay}s)"


async def demo_timeouts() -> None:
    """Demonstrate timeout behavior with various delays."""
    print("=== Individual Timeouts ===")
    tasks: list[tuple[str, float, float]] = [
        ("Fast API", 0.1, 1.0),    # Completes well within timeout
        ("Medium API", 0.5, 1.0),   # Completes within timeout
        ("Slow API", 2.0, 0.5),     # Exceeds timeout
        ("Dead API", 10.0, 0.3),    # Way over timeout
    ]

    results = await asyncio.gather(
        *[fetch_with_timeout(name, delay, timeout) for name, delay, timeout in tasks]
    )

    for result in results:
        status = "TIMEOUT" if "TIMED OUT" in result else "OK"
        print(f"  [{status}] {result}")

    # Timeout on the entire gather operation
    print("\n=== Timeout on Entire Batch ===")
    try:
        batch_results = await asyncio.wait_for(
            asyncio.gather(
                slow_network_call("A", 0.1),
                slow_network_call("B", 0.2),
                slow_network_call("C", 5.0),  # This one is too slow
            ),
            timeout=0.5,
        )
        print(f"  All completed: {batch_results}")
    except asyncio.TimeoutError:
        print("  Entire batch timed out (one slow call held up the group)")


await demo_timeouts()

## JSON Message Framing: Length-Prefixed Protocol

When sending structured data (like JSON) over TCP, we need a **framing protocol**
to know where one message ends and the next begins. A common approach is
**length-prefixed framing**:

1. Send a 4-byte big-endian integer indicating the message length
2. Send the JSON payload of exactly that length

This is more robust than newline-delimited framing because JSON can contain
newlines within string values.

In [ ]:
import asyncio
import json
import struct
from typing import Any


async def send_json(
    writer: asyncio.StreamWriter,
    data: dict[str, Any],
) -> None:
    """Send a JSON message with a 4-byte length prefix."""
    payload: bytes = json.dumps(data, ensure_ascii=False).encode("utf-8")
    header: bytes = struct.pack("!I", len(payload))
    writer.write(header + payload)
    await writer.drain()


async def recv_json(
    reader: asyncio.StreamReader,
) -> dict[str, Any]:
    """Receive a length-prefixed JSON message."""
    header: bytes = await reader.readexactly(4)
    (length,) = struct.unpack("!I", header)
    payload: bytes = await reader.readexactly(length)
    return json.loads(payload.decode("utf-8"))


async def json_rpc_server(
    reader: asyncio.StreamReader,
    writer: asyncio.StreamWriter,
) -> None:
    """A simple JSON-RPC-like server."""
    addr = writer.get_extra_info("peername")
    print(f"[JSON Server] Client connected from {addr}")

    try:
        while True:
            try:
                request: dict[str, Any] = await recv_json(reader)
            except asyncio.IncompleteReadError:
                break  # Client disconnected

            method: str = request.get("method", "unknown")
            params: dict[str, Any] = request.get("params", {})
            req_id: int = request.get("id", 0)

            # Simple method dispatch
            if method == "add":
                result = params.get("a", 0) + params.get("b", 0)
            elif method == "greet":
                result = f"Hello, {params.get('name', 'World')}!"
            elif method == "echo":
                result = params
            else:
                result = {"error": f"Unknown method: {method}"}

            response: dict[str, Any] = {"id": req_id, "result": result}
            await send_json(writer, response)
    finally:
        writer.close()
        await writer.wait_closed()


async def demo_json_protocol() -> None:
    """Demonstrate length-prefixed JSON messaging."""
    server = await asyncio.start_server(json_rpc_server, "127.0.0.1", 0)
    port: int = server.sockets[0].getsockname()[1]
    print(f"JSON-RPC server on port {port}\n")

    reader, writer = await asyncio.open_connection("127.0.0.1", port)

    # Send various requests
    requests: list[dict[str, Any]] = [
        {"id": 1, "method": "add", "params": {"a": 10, "b": 32}},
        {"id": 2, "method": "greet", "params": {"name": "Python"}},
        {"id": 3, "method": "echo", "params": {"data": [1, 2, 3], "nested": {"key": "value"}}},
        {"id": 4, "method": "unknown_method", "params": {}},
    ]

    for req in requests:
        await send_json(writer, req)
        resp: dict[str, Any] = await recv_json(reader)
        print(f"  Request:  {json.dumps(req)}")
        print(f"  Response: {json.dumps(resp)}\n")

    writer.close()
    await writer.wait_closed()
    server.close()
    await server.wait_closed()


await demo_json_protocol()

## Scalability: Handling Many Concurrent Connections

One of `asyncio`'s key advantages is handling thousands of concurrent connections
on a single thread. Each connection is a lightweight coroutine (not an OS thread),
so the overhead per connection is minimal. This cell demonstrates scaling to
many simultaneous clients.

In [ ]:
import asyncio
import time


request_count: int = 0


async def counting_handler(
    reader: asyncio.StreamReader,
    writer: asyncio.StreamWriter,
) -> None:
    """Simple handler that counts requests."""
    global request_count
    data: bytes = await reader.readline()
    if data:
        request_count += 1
        writer.write(b"OK\n")
        await writer.drain()
    writer.close()
    await writer.wait_closed()


async def make_request(host: str, port: int, message: str) -> bool:
    """Make a single request and return success status."""
    try:
        reader, writer = await asyncio.open_connection(host, port)
        writer.write(f"{message}\n".encode())
        await writer.drain()
        response: bytes = await reader.readline()
        writer.close()
        await writer.wait_closed()
        return response.strip() == b"OK"
    except Exception:
        return False


async def demo_scalability() -> None:
    """Show async server handling many concurrent connections."""
    global request_count
    request_count = 0

    server = await asyncio.start_server(counting_handler, "127.0.0.1", 0)
    port: int = server.sockets[0].getsockname()[1]

    # Test with increasing numbers of concurrent clients
    for num_clients in [10, 50, 100, 200]:
        request_count = 0
        start = time.perf_counter()

        results: list[bool] = await asyncio.gather(
            *[make_request("127.0.0.1", port, f"req-{i}") for i in range(num_clients)]
        )

        elapsed: float = time.perf_counter() - start
        successes: int = sum(results)
        rps: float = num_clients / elapsed if elapsed > 0 else 0

        print(
            f"  {num_clients:>4} clients: {successes}/{num_clients} succeeded, "
            f"{elapsed:.3f}s, {rps:.0f} req/s"
        )

    server.close()
    await server.wait_closed()


print("=== Async Server Scalability ===")
await demo_scalability()

## Comparison: Sync Sockets vs Async Streams

The following comparison highlights the key differences between traditional
blocking socket programming and async stream-based networking.

| Aspect | Sync Sockets | Async Streams |
|--------|-------------|---------------|
| **Module** | `socket` | `asyncio` |
| **Concurrency model** | One thread per connection | One event loop, many coroutines |
| **Connection setup** | `socket()` + `connect()` | `await open_connection()` |
| **Server setup** | `bind()` + `listen()` + `accept()` | `await start_server()` |
| **Reading** | `recv()` / `recv_into()` | `await reader.read()` / `readline()` |
| **Writing** | `sendall()` | `writer.write()` + `await drain()` |
| **Blocking** | Yes (unless non-blocking mode) | Never (cooperative yielding) |
| **Scalability** | ~100s of threads | ~10,000s of coroutines |
| **Error handling** | `socket.error`, `OSError` | `asyncio.IncompleteReadError`, etc. |

In [ ]:
import asyncio
import socket
import threading
import time


NUM_CONNECTIONS: int = 50
SIMULATED_LATENCY: float = 0.05  # 50ms per request


# --- Sync (threading) approach ---
def sync_handle_client(conn: socket.socket) -> None:
    with conn:
        data = conn.recv(1024)
        time.sleep(SIMULATED_LATENCY)  # Simulate processing
        conn.sendall(b"OK")


def run_sync_server(port: int, ready: threading.Event) -> None:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
        srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        srv.bind(("127.0.0.1", port))
        srv.listen(100)
        srv.settimeout(5.0)
        ready.set()

        threads: list[threading.Thread] = []
        for _ in range(NUM_CONNECTIONS):
            try:
                conn, _ = srv.accept()
                t = threading.Thread(target=sync_handle_client, args=(conn,), daemon=True)
                t.start()
                threads.append(t)
            except socket.timeout:
                break
        for t in threads:
            t.join(timeout=5)


def run_sync_clients(port: int) -> float:
    start = time.perf_counter()
    threads: list[threading.Thread] = []

    def client_task() -> None:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect(("127.0.0.1", port))
            s.sendall(b"hello")
            s.recv(1024)

    for _ in range(NUM_CONNECTIONS):
        t = threading.Thread(target=client_task, daemon=True)
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    return time.perf_counter() - start


# --- Async approach ---
async def async_handle_client(
    reader: asyncio.StreamReader,
    writer: asyncio.StreamWriter,
) -> None:
    await reader.read(1024)
    await asyncio.sleep(SIMULATED_LATENCY)  # Simulate processing
    writer.write(b"OK")
    await writer.drain()
    writer.close()
    await writer.wait_closed()


async def run_async_benchmark() -> float:
    server = await asyncio.start_server(async_handle_client, "127.0.0.1", 0)
    port: int = server.sockets[0].getsockname()[1]

    async def client_task() -> None:
        reader, writer = await asyncio.open_connection("127.0.0.1", port)
        writer.write(b"hello")
        await writer.drain()
        await reader.read(1024)
        writer.close()
        await writer.wait_closed()

    start = time.perf_counter()
    await asyncio.gather(*[client_task() for _ in range(NUM_CONNECTIONS)])
    elapsed = time.perf_counter() - start

    server.close()
    await server.wait_closed()
    return elapsed


# Run sync benchmark
print(f"=== Benchmark: {NUM_CONNECTIONS} Concurrent Connections ===")
print(f"    Simulated latency: {SIMULATED_LATENCY * 1000:.0f}ms per request\n")

# Find a free port for the sync server
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as tmp:
    tmp.bind(("127.0.0.1", 0))
    sync_port: int = tmp.getsockname()[1]

ready = threading.Event()
server_thread = threading.Thread(
    target=run_sync_server, args=(sync_port, ready), daemon=True
)
server_thread.start()
ready.wait()
sync_time: float = run_sync_clients(sync_port)
server_thread.join(timeout=5)
print(f"  Threading:  {sync_time:.3f}s")

# Run async benchmark
async_time: float = await run_async_benchmark()
print(f"  Asyncio:    {async_time:.3f}s")

if sync_time > 0 and async_time > 0:
    faster = "asyncio" if async_time < sync_time else "threading"
    ratio = max(sync_time, async_time) / min(sync_time, async_time)
    print(f"\n  {faster} was {ratio:.1f}x faster")
    print(f"  (Both handle the same {NUM_CONNECTIONS} concurrent connections)")

## Graceful Shutdown Pattern

Production async servers need to shut down gracefully: stop accepting new
connections, let in-flight requests complete, and then close. This pattern
uses `asyncio.Event` as a shutdown signal and `server.close()` +
`server.wait_closed()` for clean termination.

In [ ]:
import asyncio
from dataclasses import dataclass, field
from typing import Any


@dataclass
class ServerStats:
    """Track server statistics."""
    connections_accepted: int = 0
    connections_active: int = 0
    requests_processed: int = 0
    errors: int = 0


class AsyncTCPServer:
    """A production-style async TCP server with graceful shutdown."""

    def __init__(self, host: str = "127.0.0.1", port: int = 0) -> None:
        self.host: str = host
        self.port: int = port
        self.stats: ServerStats = ServerStats()
        self._shutdown_event: asyncio.Event = asyncio.Event()
        self._server: asyncio.Server | None = None

    async def handle_client(
        self,
        reader: asyncio.StreamReader,
        writer: asyncio.StreamWriter,
    ) -> None:
        """Handle a client connection."""
        self.stats.connections_accepted += 1
        self.stats.connections_active += 1
        addr = writer.get_extra_info("peername")

        try:
            while not self._shutdown_event.is_set():
                try:
                    data: bytes = await asyncio.wait_for(
                        reader.readline(), timeout=1.0
                    )
                except asyncio.TimeoutError:
                    continue  # Check shutdown flag

                if not data:
                    break

                self.stats.requests_processed += 1
                message: str = data.decode().strip()
                response: str = f"Processed: {message}\n"
                writer.write(response.encode())
                await writer.drain()

        except Exception:
            self.stats.errors += 1
        finally:
            self.stats.connections_active -= 1
            writer.close()
            await writer.wait_closed()

    async def start(self) -> int:
        """Start the server and return the assigned port."""
        self._server = await asyncio.start_server(
            self.handle_client, self.host, self.port
        )
        actual_port: int = self._server.sockets[0].getsockname()[1]
        self.port = actual_port
        print(f"Server started on {self.host}:{actual_port}")
        return actual_port

    async def shutdown(self, grace_period: float = 2.0) -> None:
        """Gracefully shut down the server."""
        if self._server is None:
            return

        print("Initiating graceful shutdown...")
        self._shutdown_event.set()  # Signal handlers to stop

        # Stop accepting new connections
        self._server.close()
        await self._server.wait_closed()

        # Wait for active connections to finish
        if self.stats.connections_active > 0:
            print(f"  Waiting for {self.stats.connections_active} active connection(s)...")
            await asyncio.sleep(grace_period)

        print(f"Server shut down. Stats: {self.stats}")


# Demonstrate graceful shutdown
async def demo_graceful_shutdown() -> None:
    server = AsyncTCPServer()
    port = await server.start()

    async def client_session(client_id: int) -> None:
        reader, writer = await asyncio.open_connection("127.0.0.1", port)
        for i in range(3):
            writer.write(f"Client-{client_id} msg-{i}\n".encode())
            await writer.drain()
            response = await reader.readline()
            await asyncio.sleep(0.05)
        writer.close()
        await writer.wait_closed()

    # Run clients
    await asyncio.gather(
        *[client_session(i) for i in range(5)]
    )

    # Graceful shutdown
    await server.shutdown(grace_period=0.5)


await demo_graceful_shutdown()

## Summary

This notebook covered async networking with Python's `asyncio` module:

1. **`asyncio.open_connection()`** for creating async TCP clients with `StreamReader`/`StreamWriter`
2. **`StreamReader` API**: `read()`, `readline()`, `readexactly()` for flexible async reads
3. **`StreamWriter` API**: `write()`, `drain()`, `close()`, `wait_closed()` for buffered sends
4. **`asyncio.start_server()`** for creating async TCP servers with per-connection handlers
5. **`asyncio.gather()`** for concurrent connections and parallel request execution
6. **`asyncio.wait_for()`** for adding timeouts to any async operation
7. **Length-prefixed JSON protocol** for structured message framing over TCP
8. **Scalability**: handling hundreds of concurrent connections on a single thread
9. **Sync vs async comparison**: threading-based sockets vs asyncio streams
10. **Graceful shutdown**: production patterns for clean server termination

Together with notebooks 01 (socket fundamentals) and 02 (HTTP and URLs), this
completes the networking toolkit: low-level sockets for protocol control,
`urllib` for HTTP/URL operations, and `asyncio` for high-performance concurrent networking.